# Проект по SQL


***АНАЛИТИКА БАЗЫ ДАННЫХ ДЛЯ КРУПНОГО КНИЖНОГО СЕРВИСА***

***ЦЕЛЬ:*** Проанализировать существующую базу данных для определение ценностного предложения для нового продукта.

***ОПИСАНИЕ ДАННЫХ:***

Таблица books
Содержит данные о книгах:

book_id — идентификатор книги;

author_id — идентификатор автора;

title — название книги;

num_pages — количество страниц;

publication_date — дата публикации книги;

publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:

author_id — идентификатор автора;

author — имя автора.

Таблица publishers
Содержит данные об издательствах:

publisher_id — идентификатор издательства;

publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:

rating_id — идентификатор оценки;

book_id — идентификатор книги;

username — имя пользователя, оставившего оценку;

rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах:

review_id — идентификатор обзора;

book_id — идентификатор книги;

username — имя автора обзора;

text — текст обзора.

<div >
    



***СХЕМА ДАННЫХ***  

![image.png](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

***ПОДКЛЮЧЕНИЕ К БАЗЕ ДАННЫХ***

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas

***ВЫВЕДЕМ ПЕРВЫЕ ПЯТЬ СТРОК БД***

In [4]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [8]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


***ЗАДАНИЯ***

1.Посчитайте, сколько книг вышло после 1 января 2000 года;

In [9]:
query = '''SELECT COUNT (DISTINCT book_id)
FROM books
WHERE publication_date > '2000-01-01' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


***ВЫВОД:*** После 1 января 2000 года вышло 819 книг.

2.Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [10]:
query = '''WITH 

reviews_count AS
(SELECT b.book_id, b.title, COUNT (rw.review_id) as rev_cnt
FROM books AS b
LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
GROUP BY b.book_id),

avg_rat AS 
(SELECT b.book_id, ROUND (AVG (r.rating),2) as avg_rat
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id)

SELECT cr.book_id, cr.title, cr.rev_cnt, av.avg_rat
FROM reviews_count AS cr
LEFT JOIN avg_rat AS av ON cr.book_id = av.book_id
ORDER BY rev_cnt DESC, avg_rat DESC '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,rev_cnt,avg_rat
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


***ВЫВОД:*** Есть более популярные и менее популярные книги, стоит отменить, не смотря на большее количетсво обзоров рейтинг сохраняется в пределах 3,6 - 4 как и у книг без обзоров.

3.Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
query = '''WITH 
not_br AS 
(SELECT (COUNT (b.book_id)) AS cnt, b.publisher_id
FROM books AS b
WHERE num_pages > 50
GROUP BY b.publisher_id
ORDER BY cnt DESC
LIMIT 1)
,

publisher AS 
(SELECT p.publisher_id, p.publisher
FROM publishers AS p)

SELECT pb.publisher, cnt
FROM not_br AS nb
LEFT JOIN publisher AS pb ON nb.publisher_id = pb.publisher_id
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,cnt
0,Penguin Books,42


***ВЫВОД:*** Изадтельство  Penguin Books выпустило наибольшее количество книг (42) толще 50 страниц.

4.Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [12]:
query ='''WITH 

rating_b AS
(SELECT COUNT (rating_id) AS cnt,book_id, ROUND (AVG (rating), 2) AS z
FROM ratings
GROUP BY book_id
HAVING COUNT (rating_id) >=50
ORDER BY AVG (rating) DESC),

books_a AS 
(SELECT book_id, author_id, title
FROM books),

author_a AS 
(SELECT author_id, author
FROM authors)

SELECT  a.author, ROUND (AVG (rb.z),3)
FROM rating_b AS rb
LEFT JOIN books AS b ON rb.book_id = b.book_id
LEFT JOIN author_a AS a ON b.author_id = a.author_id
GROUP BY a.author
LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.285


***ВЫВОД:*** J.K. Rowling/Mary GrandPré является авторами  с наибольшей средней оценкой среди книг с 50 и более оценками.

5.Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query ='''WITH

users_rat AS (SELECT COUNT (rating_id), username
FROM ratings
GROUP BY  username
HAVING COUNT (rating_id) > 48),

users_rev AS (SELECT COUNT(review_id) AS cnt, username
FROM reviews
GROUP BY username)

SELECT  AVG(ure.cnt)
FROM  users_rat AS ura
LEFT JOIN users_rev AS ure ON ura.username = ure.username'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


***ВЫВОД:*** Среднее количество обзоров 24 у пользователей, которые поставили более 48 оценок.

***ФИНАЛЬНЫЙ ВЫВОД: 
После проведенного исследования базы данных, можно сделать вывод о предпочтениях пользователей, более популярными книгами являются Twilight (Сумерки) и  серия книг про Harry Potter, но не смотря на большое количество обзоров рейтинг сохраняется в пределах 3,6 - 4, как и у книг без обзоров. Наиболее популярным автором является автор Harry Potter- J.K. Rowling/Mary GrandPré. Издательство, которое выпустило наибольшее количество книг (42) толще 50 страниц Penguin Books.  Среднее количество обзоров пользователей составило 24 обзора (у пользователей с оценками  48+ ). Указанную информацию можно использовать для нового продукта (книжный сервис), например, размещать наиболее популярных авторов и их издания.***